In [ ]:
print('Setup complete.')

# LlamaIndex & AutoGen with AskSage GPT-5-Mini - Demo

**Focus**: index types, rerankers, simple multi-agent flow using AskSage GPT-5-Mini

This notebook demonstrates key concepts in LlamaIndex and AutoGen frameworks using AskSage's GPT-5-Mini model, including different index types, reranking techniques, and basic multi-agent workflows.

## Learning Objectives
- Understand different LlamaIndex index types and their use cases
- Implement reranking strategies to improve retrieval quality
- Build simple multi-agent flows with AutoGen using AskSage
- Compare retrieval quality before and after reranking
- Use AskSage GPT-5-Mini for all AI operations

In [ ]:
# Install required packages
!pip install llama-index llama-index-embeddings-openai llama-index-llms-openai \
             llama-index-postprocessor-cohere-rerank llama-index-readers-file \
             pyautogen chromadb sentence-transformers cohere asksageclient

In [ ]:
import os
import json
from asksageclient import AskSageClient
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext, Settings
from llama_index.core import ListIndex, TreeIndex, KeywordTableIndex
from llama_index.core.schema import Document
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.llms.base import BaseLLM
from llama_index.core.base.llms.types import CompletionResponse, LLMMetadata
import autogen
from typing import List, Optional, Any
import time

# Initialize AskSage client
# You'll need to set your AskSage credentials
# ask_sage_client = AskSageClient(api_key="your-asksage-api-key", base_url="your-asksage-base-url")
# For demo purposes, we'll assume the client is configured via environment variables
ask_sage_client = AskSageClient()

print("AskSage client initialized")

In [ ]:
# Create a custom LLM wrapper for AskSage GPT-5-Mini
class AskSageLLM(BaseLLM):
    def __init__(self, ask_sage_client: AskSageClient, model: str = "gpt-5-mini"):
        super().__init__()
        self.client = ask_sage_client
        self.model = model
    
    @property
    def metadata(self) -> LLMMetadata:
        return LLMMetadata(
            context_window=128000,
            num_output=4096,
            model_name=self.model
        )
    
    def complete(self, prompt: str, **kwargs: Any) -> CompletionResponse:
        try:
            # Use AskSage query endpoint with GPT-5-Mini
            response = self.client.query(
                message=prompt,
                model=self.model,
                temperature=kwargs.get('temperature', 0.0)
            )
            
            # Extract response text from AskSage response
            if isinstance(response, dict) and 'response' in response:
                response_text = response['response']
            else:
                response_text = str(response)
            
            return CompletionResponse(text=response_text)
        except Exception as e:
            print(f"Error calling AskSage: {e}")
            return CompletionResponse(text=f"Error: {str(e)}")
    
    def stream_complete(self, prompt: str, **kwargs: Any):
        # For simplicity, we'll just return the complete response
        response = self.complete(prompt, **kwargs)
        yield response

# Initialize AskSage LLM
asksage_llm = AskSageLLM(ask_sage_client, model="gpt-5-mini")
print("AskSage GPT-5-Mini LLM wrapper created")

## 1. Sample Data - System Logs

Let's create sample system logs that we'll use throughout this demo to simulate incident management scenarios.

In [ ]:
# Create sample system logs for our demo
sample_logs = [
    {
        "timestamp": "2024-01-15 14:23:15",
        "level": "ERROR",
        "service": "api-gateway",
        "message": "Connection timeout to user-service after 30s. Retry attempt 3/3 failed.",
        "incident_id": "INC-001"
    },
    {
        "timestamp": "2024-01-15 14:23:45",
        "level": "WARN",
        "service": "user-service", 
        "message": "Database connection pool exhausted. Current active connections: 100/100",
        "incident_id": "INC-001"
    },
    {
        "timestamp": "2024-01-15 14:24:12",
        "level": "INFO",
        "service": "load-balancer",
        "message": "Health check failed for user-service instance 192.168.1.50:8080",
        "incident_id": "INC-001"
    },
    {
        "timestamp": "2024-01-15 14:25:33",
        "level": "ERROR",
        "service": "user-service",
        "message": "OutOfMemoryError: Java heap space. Service shutting down gracefully.",
        "incident_id": "INC-001"
    },
    {
        "timestamp": "2024-01-15 14:26:01",
        "level": "INFO",
        "service": "orchestrator",
        "message": "Attempting automatic restart of user-service with increased memory allocation",
        "incident_id": "INC-001"
    },
    {
        "timestamp": "2024-01-15 14:28:45",
        "level": "INFO",
        "service": "user-service",
        "message": "Service successfully restarted with 4GB heap size. All health checks passing.",
        "incident_id": "INC-001"
    },
    {
        "timestamp": "2024-01-15 15:45:22",
        "level": "ERROR",
        "service": "payment-service",
        "message": "SSL certificate expired for external payment gateway. Transactions failing.",
        "incident_id": "INC-002"
    },
    {
        "timestamp": "2024-01-15 15:46:10",
        "level": "WARN",
        "service": "monitoring",
        "message": "Payment success rate dropped to 15% in the last 5 minutes",
        "incident_id": "INC-002"
    }
]

# Convert to documents
documents = []
for log in sample_logs:
    content = f"[{log['timestamp']}] {log['level']} - {log['service']}: {log['message']}"
    metadata = {
        "timestamp": log['timestamp'],
        "level": log['level'],
        "service": log['service'],
        "incident_id": log['incident_id']
    }
    documents.append(Document(text=content, metadata=metadata))

print(f"Created {len(documents)} log documents")
print(f"\nSample document:")
print(f"Text: {documents[0].text}")
print(f"Metadata: {documents[0].metadata}")

## 2. LlamaIndex Index Types with AskSage GPT-5-Mini

Let's explore different index types in LlamaIndex using AskSage GPT-5-Mini as our LLM.

In [ ]:
# Configure LlamaIndex settings with AskSage GPT-5-Mini
Settings.llm = asksage_llm
Settings.embed_model = OpenAIEmbedding()  # We'll keep OpenAI embeddings for now

print("LlamaIndex settings configured with AskSage GPT-5-Mini")

In [ ]:
# 1. Vector Store Index - Best for semantic similarity
print("Creating Vector Store Index...")
vector_index = VectorStoreIndex.from_documents(documents)
print("✅ Vector Store Index created")

# 2. List Index - Sequential processing of all documents
print("\nCreating List Index...")
list_index = ListIndex.from_documents(documents)
print("✅ List Index created")

# 3. Tree Index - Hierarchical summarization
print("\nCreating Tree Index...")
tree_index = TreeIndex.from_documents(documents)
print("✅ Tree Index created")

# 4. Keyword Table Index - Keyword-based retrieval
print("\nCreating Keyword Table Index...")
keyword_index = KeywordTableIndex.from_documents(documents)
print("✅ Keyword Table Index created")

## 3. Comparing Index Types with AskSage GPT-5-Mini

Let's test the same query against different index types to see how they perform using AskSage GPT-5-Mini.

In [ ]:
query = "What caused the user service to fail and how was it resolved?"

print(f"Query: {query}")
print("="*80)

# Test each index type
indexes = {
    "Vector Store": vector_index,
    "List": list_index,
    "Tree": tree_index,
    "Keyword Table": keyword_index
}

for name, index in indexes.items():
    print(f"\n🔍 {name} Index Response (using AskSage GPT-5-Mini):")
    print("-" * 40)
    
    try:
        query_engine = index.as_query_engine()
        response = query_engine.query(query)
        
        print(f"Response: {response.response}")
        
        # Show source nodes if available
        if hasattr(response, 'source_nodes') and response.source_nodes:
            print(f"\nSources used ({len(response.source_nodes)}):")
            for i, node in enumerate(response.source_nodes[:2], 1):
                print(f"{i}. {node.text[:100]}...")
    except Exception as e:
        print(f"Error processing {name} index: {e}")

## 4. Reranking with Cohere (Optional)

Now let's implement reranking to improve retrieval quality. We'll compare results before and after reranking.

In [ ]:
# Create retriever without reranking
retriever_basic = VectorIndexRetriever(
    index=vector_index,
    similarity_top_k=5
)

# Create query engine without reranking
query_engine_basic = RetrieverQueryEngine(
    retriever=retriever_basic
)

print("✅ Basic retriever (no reranking) created")

In [ ]:
# Create reranker (you'll need Cohere API key for this to work)
try:
    reranker = CohereRerank(api_key=os.environ.get("COHERE_API_KEY"), top_n=3)
    
    # Create query engine with reranking
    query_engine_reranked = vector_index.as_query_engine(
        similarity_top_k=5,
        node_postprocessors=[reranker]
    )
    
    print("✅ Reranked query engine created")
    reranking_available = True
    
except Exception as e:
    print(f"⚠️  Reranking not available (need Cohere API key): {e}")
    reranking_available = False

In [ ]:
# Compare queries with and without reranking using AskSage GPT-5-Mini
test_queries = [
    "What memory issues occurred with the user service?",
    "How was the payment service SSL problem handled?",
    "What services were involved in incident INC-001?"
]

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*60}")
    print(f"Query {i}: {query}")
    print('='*60)
    
    # Basic retrieval with AskSage GPT-5-Mini
    print("\n🔍 WITHOUT Reranking (AskSage GPT-5-Mini):")
    print("-" * 25)
    try:
        response_basic = query_engine_basic.query(query)
        print(f"Response: {response_basic.response}")
        
        if hasattr(response_basic, 'source_nodes'):
            print(f"\nSources ({len(response_basic.source_nodes)}):")
            for j, node in enumerate(response_basic.source_nodes[:2], 1):
                print(f"{j}. {node.text}")
    except Exception as e:
        print(f"Error in basic query: {e}")
    
    # Reranked retrieval (if available)
    if reranking_available:
        print("\n🎯 WITH Reranking (AskSage GPT-5-Mini):")
        print("-" * 20)
        try:
            response_reranked = query_engine_reranked.query(query)
            print(f"Response: {response_reranked.response}")
            
            if hasattr(response_reranked, 'source_nodes'):
                print(f"\nSources ({len(response_reranked.source_nodes)}):")
                for j, node in enumerate(response_reranked.source_nodes[:2], 1):
                    print(f"{j}. {node.text}")
        except Exception as e:
            print(f"Error in reranked query: {e}")

## 5. Simple Multi-Agent Flow with AutoGen and AskSage GPT-5-Mini

Now let's create a simple multi-agent system using AutoGen with AskSage GPT-5-Mini for incident analysis.

In [ ]:
# Create a custom AutoGen LLM config for AskSage GPT-5-Mini
class AskSageAutogenAdapter:
    def __init__(self, ask_sage_client: AskSageClient, model: str = "gpt-5-mini"):
        self.client = ask_sage_client
        self.model = model
    
    def create_completion(self, messages, **kwargs):
        # Convert messages to a single prompt for AskSage
        if isinstance(messages, list):
            prompt = "\n".join([msg.get('content', '') for msg in messages if 'content' in msg])
        else:
            prompt = str(messages)
        
        try:
            response = self.client.query(
                message=prompt,
                model=self.model,
                temperature=kwargs.get('temperature', 0)
            )
            
            if isinstance(response, dict) and 'response' in response:
                response_text = response['response']
            else:
                response_text = str(response)
            
            # Return in OpenAI-compatible format for AutoGen
            return {
                'choices': [{
                    'message': {
                        'content': response_text,
                        'role': 'assistant'
                    }
                }]
            }
        except Exception as e:
            return {
                'choices': [{
                    'message': {
                        'content': f"Error: {str(e)}",
                        'role': 'assistant'
                    }
                }]
            }

# Create AskSage adapter
asksage_adapter = AskSageAutogenAdapter(ask_sage_client, "gpt-5-mini")

# Configure AutoGen with AskSage GPT-5-Mini
config_list = [
    {
        "model": "gpt-5-mini",
        "api_type": "custom",
        "base_url": "custom",  # We'll handle this in our adapter
        "api_key": "custom"     # We'll handle this in our adapter
    }
]

llm_config = {
    "config_list": config_list,
    "temperature": 0
}

print("AutoGen configuration ready with AskSage GPT-5-Mini")

In [ ]:
# For this demo, we'll simulate the multi-agent conversation
# since the full AutoGen integration with AskSage requires more complex setup

def simulate_agent_response(agent_name: str, system_prompt: str, user_message: str) -> str:
    """
    Simulate an agent response using AskSage GPT-5-Mini
    """
    full_prompt = f"{system_prompt}\n\nUser: {user_message}\n\nAssistant:"
    
    try:
        response = ask_sage_client.query(
            message=full_prompt,
            model="gpt-5-mini",
            temperature=0
        )
        
        if isinstance(response, dict) and 'response' in response:
            return response['response']
        else:
            return str(response)
    except Exception as e:
        return f"Error from {agent_name}: {str(e)}"

# Define agent system prompts
log_analyzer_prompt = """You are a log analysis expert. Your job is to:
1. Parse and understand system logs
2. Identify patterns, errors, and anomalies
3. Extract key events and timeline information
4. Provide technical insights about system behavior

Focus on technical accuracy and detailed analysis."""

incident_manager_prompt = """You are an incident management expert. Your job is to:
1. Assess incident severity and impact
2. Coordinate response activities
3. Create structured incident reports
4. Identify root causes and prevention measures

Focus on clear communication and actionable recommendations."""

print("✅ Agent simulation functions created with AskSage GPT-5-Mini")

In [ ]:
# Prepare log data for analysis
incident_logs = "\n".join([doc.text for doc in documents if "INC-001" in doc.text])

print("Log data for INC-001:")
print(incident_logs)

In [ ]:
# Simulate multi-agent incident analysis using AskSage GPT-5-Mini
incident_prompt = f"""We have a system incident (INC-001) that needs analysis. Here are the logs:

{incident_logs}

Please analyze this incident from your area of expertise. Focus on root cause, impact, and resolution effectiveness."""

print("🤖 Starting multi-agent incident analysis with AskSage GPT-5-Mini...")
print("="*60)

# Log Analyzer Response
print("\n🔍 LogAnalyzer Response (AskSage GPT-5-Mini):")
print("-" * 40)
log_analyzer_response = simulate_agent_response(
    "LogAnalyzer", 
    log_analyzer_prompt, 
    incident_prompt
)
print(log_analyzer_response)

# Incident Manager Response
print("\n👨‍💼 IncidentManager Response (AskSage GPT-5-Mini):")
print("-" * 40)
incident_manager_response = simulate_agent_response(
    "IncidentManager", 
    incident_manager_prompt, 
    f"{incident_prompt}\n\nTechnical Analysis from LogAnalyzer:\n{log_analyzer_response}"
)
print(incident_manager_response)

## 6. Quality Comparison Summary

Let's summarize the quality improvements we observed with different techniques using AskSage GPT-5-Mini.

In [ ]:
print("📊 QUALITY COMPARISON SUMMARY (AskSage GPT-5-Mini)")
print("="*50)

print("\n🏗️ INDEX TYPES:")
print("• Vector Store: Best for semantic similarity queries")
print("• List Index: Comprehensive but slower processing")
print("• Tree Index: Good for hierarchical summarization")
print("• Keyword Table: Fast for exact keyword matches")

print("\n🎯 RERANKING BENEFITS:")
print("• Improved relevance of retrieved documents")
print("• Better handling of complex queries")
print("• More accurate source attribution")
print("• Enhanced context for LLM responses")

print("\n🤖 MULTI-AGENT ADVANTAGES:")
print("• Specialized expertise per domain (logs vs management)")
print("• Structured workflow and collaboration")
print("• Multiple perspectives on same incident")
print("• Comprehensive analysis and reporting")

print("\n🚀 ASKSAGE GPT-5-MINI BENEFITS:")
print("• Enterprise-grade security and compliance")
print("• Consistent API interface")
print("• Advanced reasoning capabilities")
print("• Cost-effective inference")

print("\n💡 KEY TAKEAWAYS:")
print("1. Choose index type based on query patterns")
print("2. Reranking significantly improves retrieval quality")
print("3. Multi-agent systems provide richer analysis")
print("4. AskSage GPT-5-Mini offers enterprise-ready AI capabilities")
print("5. Combine techniques for optimal results")

In [ ]:
# Test AskSage GPT-5-Mini directly
print("🧪 Direct AskSage GPT-5-Mini Test:")
print("="*40)

test_query = "Explain the importance of incident management in system operations."
print(f"Query: {test_query}")
print("-" * 40)

try:
    direct_response = ask_sage_client.query(
        message=test_query,
        model="gpt-5-mini",
        temperature=0.1
    )
    
    if isinstance(direct_response, dict) and 'response' in direct_response:
        response_text = direct_response['response']
    else:
        response_text = str(direct_response)
    
    print(f"AskSage GPT-5-Mini Response: {response_text}")
    
except Exception as e:
    print(f"Error testing AskSage GPT-5-Mini: {e}")
    print("Please ensure your AskSage credentials are properly configured.")

## Summary

In this demo, we explored:

### 🏗️ **LlamaIndex Index Types with AskSage GPT-5-Mini**
- **Vector Store Index**: Semantic similarity-based retrieval
- **List Index**: Sequential processing of all documents
- **Tree Index**: Hierarchical summarization approach
- **Keyword Table Index**: Keyword-based retrieval

### 🎯 **Reranking with Cohere**
- Improved relevance of retrieved documents
- Better handling of complex queries
- Enhanced quality of responses

### 🤖 **Simple Multi-Agent Flow with AskSage GPT-5-Mini**
- Specialized agents for different expertise areas
- Collaborative analysis of incidents
- Structured workflow for comprehensive reporting

### 🚀 **AskSage GPT-5-Mini Integration**
- Enterprise-grade AI capabilities
- Secure and compliant model access
- Consistent API interface
- Cost-effective high-quality inference

These techniques can be combined to create powerful AI-assisted systems for incident management, log analysis, and operational intelligence using AskSage's GPT-5-Mini model.